<a href="https://colab.research.google.com/github/dragonsan17/crime_det/blob/master/crime_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import nltk 
import string
import re
nltk.download('stopwords')

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive') 

In [0]:
from zipfile import ZipFile 
file_name = "/content/gdrive/My Drive/filtered_tweets_JK.json.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')

In [0]:
eng_tweets = []
for tweet in tweets:
  if(tweet['lang']=="en"):
    eng_tweets.append(tweet['text'])
print("appended ",len(eng_tweets)," tweets")

In [0]:
#Block : clean text, return tokenized text
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
ps = nltk.PorterStemmer()
def clean_token(text):
    #tokenize
    text = re.split('\W+', text)
    #remove stop words
    text = [word for word in text if word not in stopword]
    text = [word for word in text if word is not '']
    #stemming
    text = [ps.stem(word) for word in text]
    return text

def clean_text(eng_tweets):
  tokens = []
  for i in range(len(eng_tweets)):
      #remove handles, urls, RTs, emojis
      eng_tweets[i] = " ".join([word for word in eng_tweets[i].split()
                                  if 'http' not in word and '@' not in word and '<' not in word])
      eng_tweets[i] = eng_tweets[i].encode('ascii', 'ignore').decode('ascii')
      #remove punctuations
      for punct in '.,"':
        eng_tweets[i] = eng_tweets[i].replace(punct, ' ')
      eng_tweets[i] = remove_punct(eng_tweets[i])
      #clean token
      tokens.append(clean_token(eng_tweets[i]))
  return tokens

tokens = clean_text(eng_tweets)

In [92]:
crimes = ['hostage','murder','robbery','hostage','con','fraud','kidnap','rape','terrorism','terror','crime','protest','pelt','arson']
cnt=0
for token in tokens:
  if any(word in crimes for word in token):
    cnt+=1
print(cnt)

963
